Snapshot `__builtins__` before executing `from stdlb import *`

In [1]:
before = { k: getattr(__builtins__, k) for k in dir(__builtins__) }
before

{'ArithmeticError': ArithmeticError,
 'AssertionError': AssertionError,
 'AttributeError': AttributeError,
 'BaseException': BaseException,
 'BlockingIOError': BlockingIOError,
 'BrokenPipeError': BrokenPipeError,
 'BufferError': BufferError,
 'BytesWarning': BytesWarning,
 'ChildProcessError': ChildProcessError,
 'ConnectionAbortedError': ConnectionAbortedError,
 'ConnectionError': ConnectionError,
 'ConnectionRefusedError': ConnectionRefusedError,
 'ConnectionResetError': ConnectionResetError,
 'DeprecationWarning': DeprecationWarning,
 'EOFError': EOFError,
 'Ellipsis': Ellipsis,
 'EnvironmentError': OSError,
 'Exception': Exception,
 'False': False,
 'FileExistsError': FileExistsError,
 'FileNotFoundError': FileNotFoundError,
 'FloatingPointError': FloatingPointError,
 'FutureWarning': FutureWarning,
 'GeneratorExit': GeneratorExit,
 'IOError': OSError,
 'ImportError': ImportError,
 'ImportWarning': ImportWarning,
 'IndentationError': IndentationError,
 'IndexError': IndexError,
 '

Import most of the Python standard library, via `stdlb`

In [2]:
%%time
from stdlb import *

CPU times: user 875 µs, sys: 501 µs, total: 1.38 ms
Wall time: 1.65 ms


Snapshot `globals` after executing `from stdlb import *` (to check for inadvertent overrides of `__builtins__`)

In [3]:
after = dict(**globals())

In [4]:
both_keys = set(before).intersection(set(after))
both_keys

{'BlockingIOError',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'compile',
 'copyright',
 'get_ipython',
 'open',
 'pow'}

In [5]:
expected_diff_keys = [
    '__doc__',
    '__loader__',
    '__name__',
    '__package__',
    '__spec__',
    'get_ipython',
]

In [6]:
actual_diff_keys = []
for k, b in before.items():
    if k not in after:
        continue
    a = after[k]
    if b is not a:
        stderr.write(f'{k}: {b} != {a}\n')
        actual_diff_keys.append(k)

__doc__: Built-in functions, exceptions, and other objects.

Noteworthy: None is the `nil' object; Ellipsis represents `...' in slices. != Automatically created module for IPython interactive environment
__loader__: <class '_frozen_importlib.BuiltinImporter'> != None
__name__: builtins != __main__
__package__:  != None
__spec__: ModuleSpec(name='builtins', loader=<class '_frozen_importlib.BuiltinImporter'>, origin='built-in') != None
get_ipython: <bound method InteractiveShell.get_ipython of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x107694d30>> != <bound method InteractiveShell.get_ipython of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x107694d30>>


In [7]:
unexpected_diff_keys = set(actual_diff_keys).difference(expected_diff_keys)
if unexpected_diff_keys:
    raise RuntimeError(f"Found {len(unexpected_diff_keys)} unexpected __builtins__ overrides: {', '.join(unexpected_diff_keys)}")